In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sentence_transformers import SentenceTransformer

/home/user03/Documents/SVN/UrFU.term3/Hackaton2023/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
vk_faculty = 'Arduino программирование'
#vk_faculty = 'биофак'


coords = (53.507852, 49.420411) # latitude, longitude

In [12]:
select_query = ("select u.faculty_name_fixed_ru, u.vk_records, u.avg_cos_vk, u.vk_popularity, u.f2d_cos_rank, u.cos_edu, u.dist, u.direction,\n" +
                " u.city_rank, u.city, u.full_name\n" +
                "from \n" +
                " (select u.faculty_name_fixed_ru, u.vk_records, u.avg_cos_vk, u.vk_popularity, u.f2d_cos_rank, u.cos_edu, u.dist, u.direction,\n" +
                "   sum(u.next_city) over (partition by u.direction order by u.dist) city_rank,\n" +
                "   u.city, u.full_name\n" +
                "  from \n" +
                "   (select u.faculty_name_fixed_ru, u.vk_records, u.avg_cos_vk, u.vk_popularity, u.f2d_cos_rank, u.cos_edu, u.dist, u.direction,\n" +
                "     case when lag(city,1,'***') over (partition by u.direction order by u.dist) = city then 0 else 1 end next_city,\n" +
                "     u.city, u.full_name\n" +
                "    from\n" +
                "     (select d.faculty_name_fixed_ru, d.vk_records, d.avg_cos_vk,\n" +
                "     d.vk_popularity,\n" +
                "     d.f2d_cos_rank,\n" +
                "     d.cos_edu,\n" +
                "     get_haversine_distance(u.latitude, u.longitude, %s, %s) dist,\n" +
                "     d.direction,\n" +
                "     u.city, u.full_name\n" +
                "    from\n" +
                "     (select fr.faculty_name_fixed_ru, fr.vk_records, fr.avg_cos_vk,\n" +
                "       fv.popularity vk_popularity, -- seems it is useless\n" +
                "       row_number() over (partition by fr.faculty_name_fixed_ru order by (edu.embedding <-> fv.embedding)) f2d_cos_rank,\n" +
                "       (edu.embedding <-> fv.embedding) cos_edu, --for debug reasons\n" +
                "       edu.value as direction, edu.direction_id\n" +
                "      from\n" +
                "       (select --f.*, f2r.*\n" +
                "         f2r.faculty_name_fixed_ru, sum(vk_records) vk_records, sum(vk_records*avg_cos_vk)/sum(vk_records) as avg_cos_vk\n" +
                "        from\n" +
                "         (select d.faculty_name, count(a.record_id) vk_records, avg(cos_vk) avg_cos_vk\n" +
                "          from\n" +
                "           (select record_id, direction_id, (embedding <-> %s) cos_vk\n" +
                "            from h_additional_data\n" +
                "            order by 3 limit %s) a, -- limit number of vk people in search\n" +
                "           h_education_direction d\n" +
                "          where d.direction_id = a.direction_id\n" +
                "          group by d.faculty_name) f,\n" +
                "         h_faculty_2_ru f2r\n" +
                "        where f2r.faculty_name = f.faculty_name\n" +
                "        group by f2r.faculty_name_fixed_ru\n" +
                "        order by sum(vk_records) desc\n" +
                "        limit %s) fr, -- limit number of translated faculties selected by majority of similar vk people\n" +
                "       h_faculties_ru fv, h_edu_direction edu\n" +
                "      where fv.faculty_name_fixed_ru = fr.faculty_name_fixed_ru) d,\n" +
                "     h_edu_direct2univ d2u, h_edu_universities u\n" +
                "    where d.f2d_cos_rank <= %s -- limit number of directions selected by vectors based of faculty (can be hardcoded low value 1-2-3?)\n" +
                "     and d2u.direction_id = d.direction_id and u.university_id = d2u.university_id\n" +
                "    order by d.vk_records desc, avg_cos_vk, cos_edu, 7) u) u) u\n" +
                "where u.city_rank <= %s -- limit number of nearest cities for each faculty")

In [13]:
limit1_vk_people_cnt = 11 # limit number of vk people in search
limit2_vk_faculties_cnt = 2 # limit number of translated faculties selected by majority of similar vk people
limit3_edu_directions_for_vector = 2 # limit number of directions selected by vectors based of faculty (can be hardcoded low value 1-2-3?)
limit4_nearest_cities_for_direction = 4 # limit number of nearest cities for each faculty

In [14]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [15]:
embeddings4select = embedding_model.encode(vk_faculty)
embeddings4select = '[' + (','.join(map(str, embeddings4select))) + ']'

In [16]:
conn = psycopg2.connect(dbname="h_core", user="h_user", password="AUdZuMOGuP", host="84.201.178.7", port="55432")

In [17]:
with conn.cursor() as curs:
    curs.execute(select_query,
                 (coords[0], coords[1], embeddings4select,
                  limit1_vk_people_cnt, limit2_vk_faculties_cnt, limit3_edu_directions_for_vector, limit4_nearest_cities_for_direction))
    rs = curs.fetchall()

In [18]:
rs

[('Автоматизация и контроль',
  Decimal('1'),
  4.087086153224195,
  53,
  1,
  1.4768683707096126,
  1193.036022170363,
  'Автоматика и управление',
  1,
  'Нальчик',
  'федеральное государственное бюджетное образовательное учреждение высшего образования «Кабардино-Балкарский государственный аграрный университет имени В.М. Кокова»'),
 ('Автоматизация и контроль',
  Decimal('1'),
  4.087086153224195,
  53,
  2,
  3.4518258943644424,
  2.4884344874554873,
  'Машиностроение',
  1,
  'Тольятти',
  'федеральное государственное бюджетное образовательное учреждение высшего образования «Поволжский государственный университет сервиса»'),
 ('Автоматизация и контроль',
  Decimal('1'),
  4.087086153224195,
  53,
  2,
  3.4518258943644424,
  2.4884344874554873,
  'Машиностроение',
  1,
  'Тольятти',
  'федеральное государственное бюджетное образовательное учреждение высшего образования «Тольяттинский государственный университет»'),
 ('Автоматизация и контроль',
  Decimal('1'),
  4.087086153224195,

In [19]:
conn.close()